In [7]:
import numpy as np
# get a line of raw bitmap and plot the components
import serial
ser = serial.Serial('COM4',230400) # the name of your Pico port
print('Opening port: ')
print(ser.name)

ser.write(b'hi\r\n') # send a newline to request data
data_read = ser.read_until(b'\n',50) # read the echo

sampnum = 0
index = 0
raw = []
reds = []
greens = []
blues = []
bright = []

# Pico sends back index and raw pixel value
while sampnum < 60: # width of bitmap
    data_read = ser.read_until(b'\n',50) # read until newline
    data_text = str(data_read,'utf-8') # convert bytes to string
    data = list(map(int,data_text.split())) # convert string to values

    if(len(data)==2):
        index = data[0]
        raw.append(data[1])
        reds.append(((data[1]>>5)&0x3F)/0x3F*100) # red value is middle 6 bits
        greens.append((data[1]&0x1F)/0x1F*100) # green value is rightmost 5 bits
        blues.append(((data[1]>>11)&0x1F)/0x1F*100) # blue vale is leftmost 5 bits
        bright.append((data[1]&0x1F)+((data[1]>>5)&0x3F)+((data[1]>>11)&0x1F)) # sum of colors
        sampnum = sampnum + 1

# print the raw color as a 16bit binary to double check bitshifting
for i in range(len(reds)):
    print(f"{raw[i]:#018b}")

# plot the colors 
import matplotlib.pyplot as plt 
x = range(len(reds)) # time array
plt.plot(x,reds,'r.-',x,greens,'g.-',x,blues,'b.-')
plt.ylabel('color')
plt.xlabel('position')
plt.show()

# Threshold 
def COMf(data):
    COM2 = 0
    for i in range(60):
        COM2 += (data[i]*i) / np.sum(data)
    return COM2

conv = np.convolve(np.convolve(reds,greens,mode='same'),blues,mode='same')

c1 = np.convolve(reds,greens,mode='full')[29:90-1]
c2 = np.convolve(reds,greens,mode='same')

plt.plot(conv)
print(COMf(conv))
plt.show()

grayImage = np.zeros(60)
for i in range(60):
        grayImage[i] = int(reds[i]*0.2126 + greens[i]*0.7152 + blues[i]*0.0722)
for i in range(60):
    if grayImage[i] < 30:
        grayImage[i] = 0
    elif grayImage[i] > 60:
        grayImage[i] = 80

plt.plot(x,grayImage,'g.-')
plt.show()

# be sure to close the port
ser.close()

Opening port: 
COM4


ValueError: invalid literal for int() with base 10: '31.1129'